In [88]:
import os
import numpy as np
import matplotlib.pyplot as plt

from scipy.io import loadmat

import keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.client import device_lib
from tensorflow_addons.layers import InstanceNormalization


def check_tensorflow_gpu():
    sys_info = tf.sysconfig.get_build_info()
    cuda_version, cudnn_version = sys_info['cuda_version'], sys_info['cudnn_version']
    num_gpu_avail = len(tf.config.experimental.list_physical_devices('GPU'))
    gpu_name = device_lib.list_local_devices()[1].physical_device_desc[17:40]
    print('... Checking Tensorflow Version ...')
    print('Tensorflow built with CUDA?',  tf.test.is_built_with_cuda())
    print("TF: {} | CUDA: {} | cuDNN: {}".format(tf.__version__, cuda_version, cudnn_version))
    print('# GPU available: {} ({})'.format(num_gpu_avail, gpu_name))
    print(tf.config.list_physical_devices())
    return None
#check_tensorflow_gpu()

print('Current Directory: {}'.format(os.getcwd()))

Current Directory: /mnt/e/H2-Proxy/heterogeneity


In [4]:
'''
data_folder = 'h2datag'
file_list = [file for file in os.listdir(data_folder) if file.endswith('.mat')]
print(len(file_list))

poro = np.zeros((256,256))
perm = np.zeros((256,256))
pres = np.zeros((60,256,256))
satu = np.zeros((60,256,256))
time = np.zeros((256,256))

for i in range(1,1001):
    datax = loadmat('{}/{}UHSS_0.mat'.format(data_folder,i), simplify_cells=True)
    poro = datax['PORO']
    perm = datax['PERMX']
    for j in range(1,61):
        data = loadmat('{}/{}UHSS_{}.mat'.format(data_folder,i,j), simplify_cells=True)
        pres[j-1,:,:] = data['PRESSURE']
        satu[j-1,:,:] = data['YMF_3']*data['SGAS']
        time = np.ones((256,256))*j
    np.savez('Gdataset/sample_{}.npz'.format(i), poro=poro, perm=perm, pres=pres, sat=satu, time=time)
'''

"\ndata_folder = 'h2datag'\nfile_list = [file for file in os.listdir(data_folder) if file.endswith('.mat')]\nprint(len(file_list))\n\nporo = np.zeros((256,256))\nperm = np.zeros((256,256))\npres = np.zeros((60,256,256))\nsatu = np.zeros((60,256,256))\ntime = np.zeros((256,256))\n\nfor i in range(1,1001):\n    datax = loadmat('{}/{}UHSS_0.mat'.format(data_folder,i), simplify_cells=True)\n    poro = datax['PORO']\n    perm = datax['PERMX']\n    for j in range(1,61):\n        data = loadmat('{}/{}UHSS_{}.mat'.format(data_folder,i,j), simplify_cells=True)\n        pres[j-1,:,:] = data['PRESSURE']\n        satu[j-1,:,:] = data['YMF_3']*data['SGAS']\n        time = np.ones((256,256))*j\n    np.savez('Gdataset/sample_{}.npz'.format(i), poro=poro, perm=perm, pres=pres, sat=satu, time=time)\n"

***

In [5]:
BATCH_SIZE     = 32
INPUT_SHAPE    = (256,256,2)
IMAGE_SIZE     = 256

AUTO           = tf.data.AUTOTUNE
LAYER_NORM_EPS = 1e-6

PATCH_SIZE     = (4,4,4)
NUM_PATCHES    = (IMAGE_SIZE // PATCH_SIZE[0]) **2
PROJECTION_DIM = 128
NUM_HEADS      = 16
NUM_LAYERS     = 8

EPOCHS         = 50
LEARNING_RATE  = 1e-4
WEIGHT_DECAY   = 1e-5

In [6]:
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(filters=embed_dim, kernel_size=patch_size, strides=patch_size, padding="same")
        self.flatten    = layers.Reshape(target_shape=(-1, embed_dim))
    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(input_dim=num_tokens, output_dim=self.embed_dim)
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)
    def call(self, encoded_tokens):
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens    = encoded_tokens + encoded_positions
        return encoded_tokens

def create_vivit_model(tubelet_embedder=TubeletEmbedding(embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE), 
                       positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM), 
                       input_shape=INPUT_SHAPE, transformer_layers=NUM_LAYERS, num_heads=NUM_HEADS, embed_dim=PROJECTION_DIM, 
                       layer_norm_eps=LAYER_NORM_EPS, attn_dropout=0.1, st_dropout=0.1):
    K.clear_session()
    def decoder_layer(inp, filters, kernel_size=3, pad='same'):
        x = layers.SeparableConv2D(filters, kernel_size=kernel_size, padding=pad)(inp)
        x = layers.LayerNormalization(epsilon=layer_norm_eps)(x)
        x = layers.PReLU()(x)
        x = layers.Conv2DTranspose(filters, kernel_size=kernel_size, padding=pad, strides=2)(x)
        x = layers.SpatialDropout2D(st_dropout)(x)
        return x

    model_inputs    = layers.Input(shape=input_shape, name='model_input')
    inputs          = tf.expand_dims(model_inputs, 1)
    patches         = tubelet_embedder(inputs)
    encoded_patches = positional_encoder(patches)
    
    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim//num_heads, dropout=attn_dropout)(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=layer_norm_eps)(x2)
        x3 = keras.Sequential([layers.Dense(units=embed_dim*4, activation=tf.nn.gelu),
                               layers.Dense(units=embed_dim,   activation=tf.nn.gelu)])(x3)
        encoded_patches = layers.Add()([x3, x2])
        
    time_input = layers.Input(shape=(None,1), name='time_input')
    time_input = tf.ones(shape=encoded_patches.shape[1:])*time_input
    latent = layers.Concatenate()([encoded_patches, time_input])
    
    latent_shape = (IMAGE_SIZE//PATCH_SIZE[0], IMAGE_SIZE//PATCH_SIZE[0], latent.shape[-1])
    _ = layers.Reshape(target_shape=(latent_shape))(latent)
    _ = decoder_layer(_, 16)
    _ = decoder_layer(_, 4)
        
    outputs = layers.Conv2D(2, 3, padding='same', activation='sigmoid')(_)
    model = keras.Model(inputs=[model_inputs,time_input], outputs=outputs)
    return model

In [8]:
model = create_vivit_model()
print('# Parameters: {:,} | Output shape: {}'.format(model.count_params(), model.layers[-1].output_shape))
model.summary()

# Parameters: 2,267,258 | Output shape: (None, 256, 256, 2)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 model_input (InputLayer)    [(None, 256, 256, 2)]        0         []                            
                                                                                                  
 tf.expand_dims (TFOpLambda  (None, 1, 256, 256, 2)       0         ['model_input[0][0]']         
 )                                                                                                
                                                                                                  
 tubelet_embedding (Tubelet  (None, 4096, 128)            16512     ['tf.expand_dims[1][0]']      
 Embedding)                                                                                       
                                  

***

In [106]:
dataset_path = 'Fdataset/'
file_list = [f for f in os.listdir(dataset_path) if f.endswith('.npz')]

np.random.shuffle(file_list)

train_files = file_list[:750]
test_files  = file_list[750:]

timesteps   = np.arange(1,61)
print(timesteps.shape)

(60,)


In [98]:
def run_experiment():
    model = create_vivit_model()

    model.compile(optimizer=keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY),
                  loss=keras.losses.MeanSquaredError(name='mse'),
                  metrics=[keras.metrics.MeanSquaredError(name="mse"),
                           keras.metrics.MeanAbsolutePercentageError(name="mape")])

    history = model.fit([X_train, timesteps], y_train,
                        epochs           = EPOCHS,
                        validation_split = 0.2,
                        shuffle          = True,
                        verbose          = 0)
    _, mse, mape = model.evaluate(testloader)
    print('Test: MSE:{:.4f} | MAPE={:.4f}'.format(mse, mape))
    return model, history

model, history = run_experiment()

NameError: name 'X_train' is not defined

***
# END